### json 불러와서 저장하기

In [1]:
import numpy as np
import pandas as pd
import json
import jpype
from bs4 import BeautifulSoup as bs

data_path = "data"

json_train=[]
for i in range(0,18):
    json_file = json.loads(open("%s/%d.json" % (data_path, i)).read())
    json_train += json_file["articles"]

data/0.json
data/1.json
data/10.json
data/11.json
data/12.json
data/13.json
data/14.json
data/15.json
data/16.json
data/17.json
data/2.json
data/3.json
data/4.json
data/5.json
data/6.json
data/7.json
data/8.json
data/9.json


### json에서 text 형태소 추출

In [2]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    text = bs(article["text"],"lxml").text
    
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "text_pos": ["%s_%s" % (first, second) for first, second in mecab.pos(text)]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train

/Library/Python/2.7/site-packages/bs4/__init__.py:198: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://blog.naver.com/kakashi08/110086751501" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://ilwar.com/free/5920
http://ilwar.com/free/6078" looks like a URL. Beautiful Soup is not an HTTP client. You should probabl

0
10000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://image.kilho.net/?pk=1443527



Free" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.youtube.com/watch?v=ByhP478ZPQI

ㄹㅇ존좋" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/


20000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://imgur.com/rBtm1LI.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://m.ilwar.com/poli/17062" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "ht

...


40000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://m.khan.co.kr/view.html?category=2&med_id=khan&artid=201401071905352&code=361102" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://ilwar.com/free/44658" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python


50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.ohmynews.com/NWS_Web/View/at_pg.aspx?CNTN_CD=A0001947743" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


,istroll,text_pos
0,False,"[일간_NNG, 워스트_NNP, 마을_NNG, 회관_NNG, 입니다_VCP+EF, ..."
1,False,"[<_SY, 일간_NNG, 워스트_NNP, >_SY, 입니다_NNB+VCP+EF, ..."
2,False,"[벌써_MAG, 열린_VV+ETM, 거_NNB, 에요_VCP+EF, !_SF, ?_SF]"
3,False,"[아햏햏_NNP, 고구마_NNG, 장사_NNG, 가_JKS, 힘들_VA, 어요_EF]"
4,False,"[ㅎ_XSA, ㄷ_NNG, ㄷ_NNG, 사스_NNP, 가_JKS]"
5,False,"[민영화_NNG, 는_JX, 가_VV, 라_EF, !_SF, 우리_NP, 는_JX,..."
6,False,"[제_NP, 가_JKS, 친_VV+ETM, 드립_NNG, 이_JKS, 세_MM, 시..."
7,False,"[신장_NNG, 팔_VV, 아_EC, 간다_VX+EC]"
8,False,"[페이스북_NNP, 디자인_NNG, 카피_NNG, 하_XSV, 고_EC, 좋_VA,..."
9,False,"[일_NR, 웤ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ_UNKNOWN, 손_NN..."


###bag of words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)

    train_data_features = vectorizer.fit_transform((labeled_train[col_name].apply(lambda x:" ".join(x)))).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    labeled_train = pd.concat([labeled_train, pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)],axis=1)
    
    return labeled_train

##LDA

In [4]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    for i in range(0,num):
        if i%10000 == 0: print(i)
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = index
    
    train = pd.concat([train,df],axis=1)
        
    return train

In [6]:
max_features = 1000
%time labeled_train = make_lda(labeled_train, 5000, 1000, "text_pos")

Make Lda..
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
CPU times: user 51min 57s, sys: 3min 3s, total: 55min 1s
Wall time: 52min 34s


### predictor, model 생성

In [7]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['text_pos', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [8]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

0.643143549702


##Result
  * text의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.637526911627
  * text의 pos를 bag of words로 변환 (feature : 1,000개) - 0.661265083065
  * text의 pos를 lda로 변환 (keep_n : 5,000개, num_topics : 1,000개) - 0.643143549702
  * bs의 파라미터를 "lxml"이 아닌 "html.parser"로 주고 pos를 bag of words로 변환 (feature : 1,000개) - 0.657726939833

In [9]:
import pickle

pickle.dump(labeled_train[pre], open("text_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)